In [1]:
# imports
import numpy as np  
import io
import pandas as pd

from textblob import TextBlob
from datetime import date
from datetime import timedelta

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

In [2]:
fin_df = pd.read_csv("final_df_2016.csv")
tweet_sent_df = pd.read_csv("../raw_data/tweeter_econ_sent.csv")


/home/raz/.pyenv/versions/3.8.12/envs/btc_sent_fc/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# clean tweet corpus
def clean_df(df, column):
    df_column = df[column]
    def replace_punct(df_column):
        return df_column.str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)",'') # clean tweet of tags, punctuation

    def lower_case(df_column):
        return df_column.str.lower()
    
    df.loc[:,f'clean_{column}'] = lower_case(replace_punct(df_column))
    df[f'clean_{column}'] = df[f'clean_{column}'].apply(lambda x: str(x).replace('\n','')) # remove newlines '\n' from stings
    
    return df

df = clean_df(tweet_sent_df,'tweet')

/tmp/ipykernel_28071/1604156072.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  return df_column.str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)",'') # clean tweet of tags, punctuation


In [4]:
# tweets features processing
df['tweet_blob'] = df['clean_tweet'].apply(lambda x: TextBlob(x))
df['tweet_polarity'] = df['tweet_blob'].apply(lambda x: x.sentiment[0])
df['tweet_subjectivity'] = df['tweet_blob'].apply(lambda x: x.sentiment[1]) # switch off subjecivity if not needed
df = df[df["clean_tweet"].str.contains("us economy")==True]
df['language'] = df['clean_tweet'].apply(lambda x: detect(x))

/tmp/ipykernel_28071/676247588.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['clean_tweet'].apply(lambda x: detect(x))


In [5]:
# return a dataframe with dates, having random hours to be used in retrieving tweets

def get_dates_df():
    
    ##### get number of days to loop through #####
    def get_days_delta():
        delta = date.today()-date(2015, 12, 31)
        return delta.days

    ##### generate creates a list of dates from today to the start day in get_days_delta() #####
    dates_to_check = [(date.today() - timedelta(i)).isoformat() for i in range(get_days_delta())] 

    ##### generate random times and concat to each date  #####
    random_time_dates = []
    for i in dates_to_check:
        h = '12' #random.randint(0, 24)
        m = '00' #random.randint(0, 60)
        s = '00' #random.randint(0, 60)

        i =  f'{i}T{h}:{m}:{s}.000Z'
        random_time_dates.append(i)
        
    return pd.DataFrame(random_time_dates,columns=['all_dates'])
get_dates_df()

,all_dates
0,2021-11-26T12:00:00.000Z
1,2021-11-25T12:00:00.000Z
2,2021-11-24T12:00:00.000Z
3,2021-11-23T12:00:00.000Z
4,2021-11-22T12:00:00.000Z
...,...
2152,2016-01-05T12:00:00.000Z
2153,2016-01-04T12:00:00.000Z
2154,2016-01-03T12:00:00.000Z
2155,2016-01-02T12:00:00.000Z


In [34]:
# merge tweets df to all dates df
df_to_date = get_dates_df().join(df.set_index('tweet_date'), on='all_dates')
tweets_sentiment_df = df_to_date.groupby('all_dates').mean()

In [35]:
tweets_sentiment_df.index = pd.to_datetime(tweets_sentiment_df.index).date
tweets_sentiment_df = tweets_sentiment_df[0:2153].fillna(0)
tweets_sentiment_df.drop(columns=["tweet_subjectivity"], inplace=True)
tweets_sentiment_df = tweets_sentiment_df.rename(columns={"tweet_polarity": "tweets_sent"})

In [36]:
fin_df = fin_df.set_index('Timestamp')
fin_df.index = pd.to_datetime(fin_df.index).date
fin_df

KeyError: "None of ['Timestamp'] are in the columns"

## merge reddit dfs

In [37]:
# iport and trim csvs to match fin_df
reddit_econ_sent_df = pd.read_csv("../raw_data/reddit_econ_sent.csv")
reddit_econ_sent_df = reddit_econ_sent_df[47:]

reddit_crypto_sent_df = pd.read_csv("../raw_data/reddit_crypto_sent.csv")
reddit_crypto_sent_df = reddit_crypto_sent_df[47:]

In [38]:
# set index as date , rename columns
reddit_crypto_sent_df = reddit_crypto_sent_df.set_index('date')
reddit_crypto_sent_df = reddit_crypto_sent_df.rename(columns={"compound": "reddit_crypto_sent"})

reddit_econ_sent_df = reddit_econ_sent_df.set_index('date')
reddit_econ_sent_df = reddit_econ_sent_df.rename(columns={"compound": "reddit_econ_sent"})

In [39]:
# convert index date(str) to date
reddit_crypto_sent_df.index = pd.to_datetime(reddit_crypto_sent_df.index).date
reddit_econ_sent_df.index = pd.to_datetime(reddit_econ_sent_df.index).date

In [40]:
fin_df

,n-transactions-per-block,difficulty,utxo-count,mvrv,nvt,avg-block-size,n-transactions-excluding-popular,n-unique-addresses,median-confirmation-time,n-payments,...,mempool-growth,mempool-size,blocks-size,hash-rate,n-transactions-total,n-payments-per-block,avg-confirmation-time,nvts,transaction-fees-usd,active_account
2016-01-01,1041.690770,1.038803e+11,3.338612e+07,1.438947,13.175163,0.571189,151110.666667,263121.000000,7.333333,NaN,...,NaN,NaN,54592.003253,7.705155e+05,1.011267e+08,NaN,NaN,6.430930,13896.175481,316489.0
2016-01-02,1027.848276,1.038803e+11,3.343719e+07,1.435261,14.288515,0.579744,146859.333333,290170.000000,7.233333,NaN,...,NaN,NaN,54670.707780,7.601179e+05,1.012783e+08,NaN,NaN,6.406957,13798.467773,419389.0
2016-01-03,1072.431759,1.038803e+11,3.348825e+07,1.431575,15.401867,0.600422,142608.000000,317219.000000,7.133333,NaN,...,NaN,NaN,54759.271865,7.497203e+05,1.014362e+08,NaN,NaN,6.382985,13975.989027,394047.0
2016-01-04,1117.015243,1.038803e+11,3.353932e+07,1.427888,16.515219,0.621101,152435.333333,344268.000000,7.905556,NaN,...,NaN,NaN,54847.835949,7.393227e+05,1.015941e+08,NaN,NaN,6.359013,14153.510280,418253.0
2016-01-05,1161.598726,1.038803e+11,3.353985e+07,1.427604,13.651511,0.641779,162262.666667,350924.000000,8.677778,NaN,...,NaN,NaN,54936.400034,7.483729e+05,1.017520e+08,NaN,NaN,6.361890,14331.031533,434786.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-18,2054.611695,2.267415e+13,7.682603e+07,2.437422,2.814664,1.270386,275592.333333,725020.000000,7.297222,695747.0,...,1412.433333,4905390.0,375623.175341,1.671497e+08,6.877094e+08,4831.576389,17.654845,4.914937,771114.080949,983663.0
2021-11-19,2078.240876,2.267415e+13,7.692242e+07,2.423386,3.531133,1.275837,262180.666667,678741.666667,7.027778,689992.0,...,1606.650000,2925761.5,375815.154429,1.667887e+08,6.880043e+08,5036.437956,18.174117,4.835233,757058.299599,968315.0
2021-11-20,2078.240876,2.267415e+13,7.701882e+07,2.409351,4.247603,1.275837,248769.000000,632463.333333,6.758333,623208.0,...,725.850000,946133.0,375815.154429,1.664277e+08,6.880043e+08,4297.986207,18.693389,4.755529,757058.299599,857699.0
2021-11-21,2078.240876,2.267415e+13,7.711521e+07,2.409351,4.247603,1.275837,248769.000000,586185.000000,6.758333,570982.0,...,2829.833333,689541.0,375815.154429,1.660667e+08,6.880043e+08,4325.621212,18.693389,4.755529,757058.299599,852963.0


In [41]:
reddit_twitter_fin_df = fin_df.join(tweets_sentiment_df,)#reddit_crypto_sent_df, reddit_econ_sent_df
reddit_twitter_fin_df = reddit_twitter_fin_df.join(reddit_crypto_sent_df)
reddit_twitter_fin_df = reddit_twitter_fin_df.join(reddit_econ_sent_df)
reddit_twitter_fin_df

,n-transactions-per-block,difficulty,utxo-count,mvrv,nvt,avg-block-size,n-transactions-excluding-popular,n-unique-addresses,median-confirmation-time,n-payments,...,hash-rate,n-transactions-total,n-payments-per-block,avg-confirmation-time,nvts,transaction-fees-usd,active_account,tweets_sent,reddit_crypto_sent,reddit_econ_sent
2016-01-01,1041.690770,1.038803e+11,3.338612e+07,1.438947,13.175163,0.571189,151110.666667,263121.000000,7.333333,NaN,...,7.705155e+05,1.011267e+08,NaN,NaN,6.430930,13896.175481,316489.0,0.000000,0.021744,0.034492
2016-01-02,1027.848276,1.038803e+11,3.343719e+07,1.435261,14.288515,0.579744,146859.333333,290170.000000,7.233333,NaN,...,7.601179e+05,1.012783e+08,NaN,NaN,6.406957,13798.467773,419389.0,0.000000,0.146609,-0.045508
2016-01-03,1072.431759,1.038803e+11,3.348825e+07,1.431575,15.401867,0.600422,142608.000000,317219.000000,7.133333,NaN,...,7.497203e+05,1.014362e+08,NaN,NaN,6.382985,13975.989027,394047.0,-0.130126,0.031489,-0.162152
2016-01-04,1117.015243,1.038803e+11,3.353932e+07,1.427888,16.515219,0.621101,152435.333333,344268.000000,7.905556,NaN,...,7.393227e+05,1.015941e+08,NaN,NaN,6.359013,14153.510280,418253.0,0.163194,0.129600,-0.012812
2016-01-05,1161.598726,1.038803e+11,3.353985e+07,1.427604,13.651511,0.641779,162262.666667,350924.000000,8.677778,NaN,...,7.483729e+05,1.017520e+08,NaN,NaN,6.361890,14331.031533,434786.0,0.226389,0.026775,0.118120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-18,2054.611695,2.267415e+13,7.682603e+07,2.437422,2.814664,1.270386,275592.333333,725020.000000,7.297222,695747.0,...,1.671497e+08,6.877094e+08,4831.576389,17.654845,4.914937,771114.080949,983663.0,-0.191667,0.129760,0.164820
2021-11-19,2078.240876,2.267415e+13,7.692242e+07,2.423386,3.531133,1.275837,262180.666667,678741.666667,7.027778,689992.0,...,1.667887e+08,6.880043e+08,5036.437956,18.174117,4.835233,757058.299599,968315.0,0.155357,0.058168,0.069516
2021-11-20,2078.240876,2.267415e+13,7.701882e+07,2.409351,4.247603,1.275837,248769.000000,632463.333333,6.758333,623208.0,...,1.664277e+08,6.880043e+08,4297.986207,18.693389,4.755529,757058.299599,857699.0,0.466667,0.083604,0.014180
2021-11-21,2078.240876,2.267415e+13,7.711521e+07,2.409351,4.247603,1.275837,248769.000000,586185.000000,6.758333,570982.0,...,1.660667e+08,6.880043e+08,4325.621212,18.693389,4.755529,757058.299599,852963.0,-0.022500,0.173188,0.206028


In [45]:
# dump df to csv
reddit_twitter_fin_df.fillna(0)
reddit_twitter_fin_df.to_csv('reddit_twitter_fin.csv')

,n-transactions-per-block,difficulty,utxo-count,mvrv,nvt,avg-block-size,n-transactions-excluding-popular,n-unique-addresses,median-confirmation-time,n-payments,...,hash-rate,n-transactions-total,n-payments-per-block,avg-confirmation-time,nvts,transaction-fees-usd,active_account,tweets_sent,reddit_crypto_sent,reddit_econ_sent
2016-01-01,1041.690770,1.038803e+11,3.338612e+07,1.438947,13.175163,0.571189,151110.666667,263121.000000,7.333333,0.0,...,7.705155e+05,1.011267e+08,0.000000,0.000000,6.430930,13896.175481,316489.0,0.000000,0.021744,0.034492
2016-01-02,1027.848276,1.038803e+11,3.343719e+07,1.435261,14.288515,0.579744,146859.333333,290170.000000,7.233333,0.0,...,7.601179e+05,1.012783e+08,0.000000,0.000000,6.406957,13798.467773,419389.0,0.000000,0.146609,-0.045508
2016-01-03,1072.431759,1.038803e+11,3.348825e+07,1.431575,15.401867,0.600422,142608.000000,317219.000000,7.133333,0.0,...,7.497203e+05,1.014362e+08,0.000000,0.000000,6.382985,13975.989027,394047.0,-0.130126,0.031489,-0.162152
2016-01-04,1117.015243,1.038803e+11,3.353932e+07,1.427888,16.515219,0.621101,152435.333333,344268.000000,7.905556,0.0,...,7.393227e+05,1.015941e+08,0.000000,0.000000,6.359013,14153.510280,418253.0,0.163194,0.129600,-0.012812
2016-01-05,1161.598726,1.038803e+11,3.353985e+07,1.427604,13.651511,0.641779,162262.666667,350924.000000,8.677778,0.0,...,7.483729e+05,1.017520e+08,0.000000,0.000000,6.361890,14331.031533,434786.0,0.226389,0.026775,0.118120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-18,2054.611695,2.267415e+13,7.682603e+07,2.437422,2.814664,1.270386,275592.333333,725020.000000,7.297222,695747.0,...,1.671497e+08,6.877094e+08,4831.576389,17.654845,4.914937,771114.080949,983663.0,-0.191667,0.129760,0.164820
2021-11-19,2078.240876,2.267415e+13,7.692242e+07,2.423386,3.531133,1.275837,262180.666667,678741.666667,7.027778,689992.0,...,1.667887e+08,6.880043e+08,5036.437956,18.174117,4.835233,757058.299599,968315.0,0.155357,0.058168,0.069516
2021-11-20,2078.240876,2.267415e+13,7.701882e+07,2.409351,4.247603,1.275837,248769.000000,632463.333333,6.758333,623208.0,...,1.664277e+08,6.880043e+08,4297.986207,18.693389,4.755529,757058.299599,857699.0,0.466667,0.083604,0.014180
2021-11-21,2078.240876,2.267415e+13,7.711521e+07,2.409351,4.247603,1.275837,248769.000000,586185.000000,6.758333,570982.0,...,1.660667e+08,6.880043e+08,4325.621212,18.693389,4.755529,757058.299599,852963.0,-0.022500,0.173188,0.206028
